In [ ]:
# https://arxiv.org/pdf/2404.00308

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install evaluate
!pip install faiss-gpu
!python -m pip install scikit-image
!pip install pillow
!pip install git+https://github.com/openai/CLIP.git
!pip install peft
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
import math
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPImageProcessor, GPT2Tokenizer, CLIPProcessor, CLIPModel, GPT2LMHeadModel, Blip2QFormerConfig, Blip2QFormerModel
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score
from nltk.translate.bleu_score import sentence_bleu
from evaluate import load
import collections
from torch.cuda.amp import autocast
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
import sys
sys.path.append('/kaggle/input/yesr/pytorch/default/1')

In [ ]:
from retriever import GIFFrameRetriever
clip_cut = GIFFrameRetriever()

In [ ]:
import clip
clipm, clipp = clip.load("ViT-B/32", device="cuda")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class GQADataset(torch.utils.data.Dataset):
    def __init__(self, final_data, clip_cut, nf, clipm, clipp, train_setting = True):
        self.questions = final_data['question']
        self.image_urls = final_data['url']
        self.answers = final_data['answer']
        self.clip_cut= clip_cut
        self.clipm = clipm
        self.clipp = clipp
        self.processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.train_setting = train_setting
        self.nf = nf

    def __len__(self):
        return len(self.image_urls)

    def __getitem__(self, idx):

        url = self.image_urls[idx]
        keyframes = self.clip_cut.retrieve_images_from_gif(url, self.clipm, self.clipp, self.questions[idx], nf)

        questions = self.questions[idx]
        answers = self.answers[idx]

        processed_keyframes = [self.processor(frame, return_tensors='pt')['pixel_values'].squeeze(0) for frame in keyframes]
        processed_keyframes = torch.stack(processed_keyframes)

        tokens, mask, c_len, t_len = self.pad_sequences(self.questions[idx], self.answers[idx], nf)

        tokens = tokens.long()
        mask = mask.long()

        sample = {'images' : processed_keyframes,
                  'tokens': tokens,
                  'mask': mask,
                  'c_len' : c_len,
                  't_len' : t_len,
                  'answers' : answers,
                  'questions' : questions}

        return sample

    def pad_sequences(self, questions, answers, nf):
        m = [
          torch.tensor(self.tokenizer.encode('question: ')),
          torch.tensor(self.tokenizer.encode(' context:')),
          torch.tensor(self.tokenizer.encode('answer ')),
          torch.tensor(self.tokenizer.encode('<|endoftext|>')),
      ]
        m_mask = [
          torch.ones(len(m[0])),
          torch.ones(len(m[1])),
          torch.ones(len(m[2])),
          torch.zeros(len(m[3]))
      ]

        if self.train_setting:
            q = torch.tensor(self.tokenizer.encode(str(questions)))
            a = torch.tensor(self.tokenizer.encode(str(answers)))

            q, q_mask, leftover_tokens = self.make_padding(32, q, question=True)

            c_len =  m[1].size(0)
            t_len = m[0].size(0) + nf + q.size(0) + m[1].size(0)

            a, a_mask, _ = self.make_padding(32, a, leftover_tokens=leftover_tokens)

            if len((a == 0).nonzero()) != 0:
                pad_start = (a == 0).nonzero()[0]
            else:
                pad_start = []

            a = torch.cat((a, m[3])) if len(pad_start) == 0 else torch.cat((a[:pad_start], m[3], a[pad_start:]))
            q = torch.cat((m[1], torch.ones(nf), m[0],  q, m[2], a))

            q_mask = torch.cat((m_mask[1], torch.ones(nf), m_mask[0],  q_mask, m_mask[2], a_mask, m_mask[3]))

            return q, q_mask, c_len, t_len
        else:
            q = torch.tensor(self.tokenizer.encode(str(questions)))
            q, q_mask, _ = self.make_padding_test_setting(32, q)

            c_len =  m[1].size(0)
            t_len = m[0].size(0) + nf + q.size(0) + m[1].size(0)

            q = torch.cat((m[1], torch.ones(nf), m[0],  q, m[2]))

            q_mask = torch.cat((m_mask[1], torch.ones(nf), m_mask[0],  q_mask, m_mask[2]))
            return q, q_mask, c_len, t_len


    def make_padding(self, max_len, tokens, question=False, leftover_tokens=0):
        padding = max_len - tokens.size(0)
        if padding > 0:
            if question:
                leftover_tokens = padding
                mask = torch.ones(tokens.size(0))
            else:
                tokens = torch.cat((tokens, torch.zeros(padding + leftover_tokens)))
                mask = torch.zeros(max_len + leftover_tokens)

        elif padding == 0:
            if question:
                mask = torch.ones(tokens.size(0))
            else:
                mask = torch.zeros(tokens.size(0) + leftover_tokens)
                tokens = torch.cat((tokens, torch.zeros(leftover_tokens)))

        elif padding < 0:
            if question:
                tokens = tokens[:max_len]
                mask = torch.ones(max_len)
            else:
                tokens = torch.cat((tokens[:max_len], torch.zeros(leftover_tokens)))
                mask = torch.zeros(max_len + leftover_tokens)

        return tokens, mask, leftover_tokens


    def make_padding_test_setting(self, max_len, tokens, do_padding=False):
        padding = max_len - tokens.size(0)
        padding_len = 0
        if padding > 0:
            if do_padding:
                mask = torch.cat((torch.ones(tokens.size(0)), torch.zeros(padding)))
                tokens = torch.cat((tokens, torch.zeros(padding)))
                padding_len = padding
            else:
                mask = torch.ones(tokens.size(0))
        elif padding == 0:
            mask = torch.ones(max_len)
        elif padding < 0:
            tokens = tokens[:max_len]
            mask = torch.ones(max_len)
        return tokens, mask, padding_len

In [ ]:
peft_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM, inference_mode=False,
  r=8,
  lora_alpha=32, lora_dropout=0.1
)

In [ ]:
class MLP(nn.Module):
    def __init__(self, sizes):
        super().__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            if i < len(sizes) - 2:
                layers.append(nn.ReLU())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
class GQAModel(nn.Module):
    def __init__(self, peft_config, nf, num_query_token=3, cross_attention_freq=2):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

        for param in self.clip.parameters():
            param.requires_grad = False

        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.llm = GPT2LMHeadModel.from_pretrained("gpt2")
        self.peft_config = peft_config
        self.llm = get_peft_model(self.llm, self.peft_config)
        self.nf=  nf

        self.clip_vision_output_dim = self.clip.config.vision_config.hidden_size

        self.mapper = MLP(sizes=(self.clip_vision_output_dim, 768, 768))
        self.dropout = nn.Dropout(0.1)
        self.Qformer, self.query_tokens = self.init_Qformer(nf)
        for param in self.Qformer.parameters():
            param.requires_grad = False

        self.llm_proj = nn.Linear(self.Qformer.config.hidden_size, self.llm.config.hidden_size)
        self.image_proj = nn.Linear(512,self.Qformer.config.encoder_hidden_size)

    def init_Qformer(self, num_query_token=3):
        qformer_config = Blip2QFormerConfig.from_pretrained("Salesforce/blip2-opt-2.7b")
        qformer_config.query_length = num_query_token

        Qformer = Blip2QFormerModel.from_pretrained("Salesforce/blip2-opt-2.7b", config=qformer_config)
        query_tokens = nn.Parameter(torch.zeros(1, num_query_token, qformer_config.hidden_size))
        query_tokens.data.normal_(mean=0.0, std=qformer_config.initializer_range)

        return Qformer, query_tokens

    def forward(self, images, tokens, mask, c_len):
        batch_size, num_frames, c, h, w = images.size()

        images_reshaped = images.view(-1, c, h, w)
        image_features = self.clip.get_image_features(images_reshaped)

        image_features = self.image_proj(image_features)
        image_features = image_features.view(batch_size, num_frames, -1)

        image_atts = torch.ones(image_features.size()[:-1], dtype=torch.long).to(images.device)
        query_tokens = self.query_tokens.expand(batch_size, -1, -1) 
        query_output = self.Qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=image_features,
            encoder_attention_mask=image_atts,
            return_dict=True,
        )

        V = self.llm_proj(query_output.last_hidden_state)

        sigma = 0.1
        rho = 0.5 + sigma * torch.randn(1).item()
        rho = torch.clamp(torch.tensor(rho), 0.3, 0.7).item()
        dynamic_mask = torch.rand((V.shape[0], V.shape[1], V.shape[2]), device=V.device) > rho

        V_masked = V * dynamic_mask.float()
        caption_features = self.llm.transformer.wte(tokens)

        caption_features_masked = caption_features
        caption_features_unmasked = caption_features

        for b in range(caption_features.shape[0]):
            caption_features_masked[b,c_len[b]:c_len[b]+self.nf,:] = V_masked[b]
        for b in range(caption_features.shape[0]):
            caption_features_masked[b,c_len[b]:c_len[b]+self.nf,:] = V[b]

        llm_output_unmasked = self.llm(inputs_embeds=caption_features_unmasked, attention_mask=mask,output_hidden_states=True)
        llm_output_masked = self.llm(inputs_embeds=caption_features_masked, attention_mask=mask, output_hidden_states=True)

        masked_out = llm_output_masked.hidden_states[-1]
        unmasked_out = llm_output_unmasked.hidden_states[-1]

        return llm_output_masked, masked_out, unmasked_out, dynamic_mask

    def generate(self, images, tokens, mask, c_len):


        batch_size, num_frames, c, h, w = images.size()

        images_reshaped = images.view(-1, c, h, w)
        image_features = self.clip.get_image_features(images_reshaped)

        image_features = self.image_proj(image_features)
        image_features = image_features.view(batch_size, num_frames, -1)

        image_atts = torch.ones(image_features.size()[:-1], dtype=torch.long).to(images.device)
        query_tokens = self.query_tokens.expand(batch_size, -1, -1)

        query_output = self.Qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=image_features,
            encoder_attention_mask=image_atts,
            return_dict=True,
        )

        V = self.llm_proj(query_output.last_hidden_state)

        embedding = self.llm.transformer.wte(tokens)

        for b in range(embedding.shape[0]):
                 embedding[b,c_len[b]:c_len[b]+self.nf,:] = V[b]

        return embedding

In [ ]:
df = pd.read_csv('/kaggle/input/dataset/data.csv')

In [ ]:
def treebank_tokenize(s):
    return TreebankWordTokenizer().tokenize(s)
def generate_beam(model, tokenizer,beam_size= 1,generated=None, entry_length=50, temperature=0.8, stop_token: str = "<|endoftext|>",):
    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        for i in range(entry_length):
            outputs = model.llm(inputs_embeds=generated)
            logits = outputs.logits

            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

            logits = logits.softmax(-1).log()
            # final_logit

            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(
                    beam_size, -1
                )
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]


            next_token_embed = model.llm.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)

            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [
        tokenizer.decode(output[: int(length)])
        for output, length in zip(output_list, seq_lengths)
    ]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    model= model.to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    bleu_avg1=0.
    f1_avg = 0.
    acc = 0.
    acc_oe = 0.
    acc_yn = 0.
    c_oe =1e-9
    c_yn =1e-9

    for batch in tqdm(dataloader, desc="Testing"):
        images, tokens, mask, c_len , t_len = batch['images'], batch['tokens'], batch['mask'], batch['c_len'], batch['t_len']
        images = images.to(device)
        tokens = tokens.to(device)
        mask = mask.to(device)
        c_len = c_len.to(device)

        with autocast(dtype=torch.float16):
            with torch.no_grad():
                embed = model.generate(images, tokens, mask, c_len)
                out_text = generate_beam(model, model.tokenizer,generated=embed,entry_length=30, temperature=1)[0]

        out_text = out_text.split("<|endoftext|>")[0]

        print('Question: ', batch['questions'])
        print('Answer: ', batch['answers'])
        print("Generated_Answer: ", out_text)

        if out_text.lower()==(batch['answers'][0]).lower():
            acc+=1

        reference = [str((batch['answers'][0]).lower())]
        candidate = [out_text]

        bleu_1 = sentence_bleu(reference[0], candidate[0], weights=(1, 0, 0, 0))
        f1_avg += compute_f1(tokenizer.encode(reference[0]),tokenizer.encode(candidate[0]))
        bleu_avg1+=bleu_1

    # print('------------')
    print("BLEU {}".format(round(bleu_avg1/len(dataloader),3)))
    print("F1 {}".format(round(f1_avg/len(dataloader),3)))
    print("Accuracy {}".format(round(acc/len(dataloader),3)))


def compute_f1(gold_toks, pred_toks):
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [ ]:
nf=5
ckpt = torch.load('/kaggle/input/weights/pytorch/default/1/checkpoint_2.pt')
model = GQAModel(peft_config, nf).to(device)
model.load_state_dict(ckpt['model_state_dict'])

In [ ]:
model.eval()

In [ ]:
test_data = df.sample(frac=0.007).reset_index()
test_dataset = GQADataset(test_data,  clip_cut, nf, clipm, clipp, train_setting = False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
evaluate(model, test_loader)